In [12]:
from ultralytics import YOLO
import os
from collections import defaultdict
import numpy as np
import pandas as pd
from datetime import datetime
import cv2

# 소분류-대분류 매핑 정의
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items', 'Audio': 'Large Waste Items', 'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items', 'Main unit': 'Large Waste Items', 'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items', 'sewing machine': 'Large Waste Items', 'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items', 'Vacuum cleaner': 'Large Waste Items', 'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items', 'Sink': 'Large Waste Items', 'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items', 'Bed': 'Large Waste Items', 'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items', 'Cabinet': 'Large Waste Items', 'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items', 'Desk': 'Large Waste Items', 'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items', 'Display stand': 'Large Waste Items', 'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items', 'Small cabinet': 'Large Waste Items', 'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items', 'TV stand': 'Large Waste Items', 'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items', 'Air conditioner': 'Large Waste Items', 'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items', 'Electric rice cooker': 'Large Waste Items', 'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items', 'Heater': 'Large Waste Items', 'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items', 'refrigerator': 'Large Waste Items', 'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items', 'Washing machine': 'Large Waste Items', 'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items', 'Bedding items': 'Large Waste Items', 'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items', 'Clothes drying rack': 'Large Waste Items', 'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items', 'Earthenware jar': 'Large Waste Items', 'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items', 'lumber': 'Large Waste Items', 'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items', 'Piano': 'Large Waste Items', 'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items', 'Stroller': 'Large Waste Items', 'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items', 'audio cabinet': 'Large Waste Items', 'suitcase': 'Large Waste Items',
    
    # PP bag은 독립적인 카테고리로 유지
    'PP bag': 'PP bag',
    
    # 일반쓰레기 (General Waste)
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    
    # CleanNet
    'CleanNet': 'CleanNet'
}

class WasteDetector:
    def __init__(self, model_path, conf_threshold=0.25):
        self.model = YOLO(model_path)
        self.conf_threshold = conf_threshold
        
        # 색상 정의 (대분류별로 다른 색상 사용)
        self.category_colors = {
            'Large Waste Items': (255, 0, 0),    # 빨강
            'PP bag': (0, 255, 0),               # 초록
            'General Waste': (0, 0, 255),        # 파랑
            'CleanNet': (255, 255, 0)            # 노랑
        }
    
    def draw_detection(self, image, box, label, confidence, color):
        """박스와 라벨 그리기"""
        x1, y1, x2, y2 = map(int, box)
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        
        # 라벨 텍스트 (대분류 + 신뢰도)
        label_text = f'{label} {confidence:.2f}'
        
        # 라벨 배경 박스 크기 계산
        (text_width, text_height), _ = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
        cv2.rectangle(image, (x1, y1 - text_height - 10), (x1 + text_width + 10, y1), color, -1)
        
        # 라벨 텍스트 그리기
        cv2.putText(image, label_text, (x1 + 5, y1 - 5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        return image

    def detect_and_classify(self, image_path, save_dir):
        """이미지에서 객체 탐지 및 대분류 수행"""
        # 결과 저장 디렉토리 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_dir = os.path.join(save_dir, f'inference_{timestamp}')
        os.makedirs(output_dir, exist_ok=True)
        
        # 이미지 로드 및 추론
        results = self.model(image_path, conf=self.conf_threshold)
        
        # 대분류별 검출 결과 저장
        major_category_detections = defaultdict(list)
        
        # 각 이미지 처리
        for i, r in enumerate(results):
            img = cv2.imread(image_path)
            if img is None:
                print(f"Error: Could not load image {image_path}")
                continue
            
            # 각 검출 결과 처리
            for box in r.boxes:
                # 원본 클래스 이름과 대분류 매핑
                orig_class = self.model.names[int(box.cls)]
                major_class = category_mapping.get(orig_class, orig_class)
                conf = float(box.conf)
                
                # 박스 좌표
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                
                # 결과 저장
                major_category_detections[major_class].append({
                    'confidence': conf,
                    'box': [x1, y1, x2, y2],
                    'original_class': orig_class
                })
                
                # 대분류 기준으로 박스와 라벨 그리기
                color = self.category_colors.get(major_class, (128, 128, 128))
                img = self.draw_detection(img, [x1, y1, x2, y2], major_class, conf, color)
            
            # 결과 이미지 저장
            output_path = os.path.join(output_dir, f'result_{i+1}.jpg')
            cv2.imwrite(output_path, img)
            
            # 결과 출력
            print(f"\nDetections in image {i+1}:")
            for major_class, detections in major_category_detections.items():
                if detections:
                    print(f"\n{major_class}:")
                    for det in detections:
                        print(f"  - Confidence: {det['confidence']:.2f}")
        
        # 메트릭스 계산 및 저장
        metrics = self.calculate_metrics(major_category_detections)
        metrics_df = pd.DataFrame(metrics).transpose()
        metrics_df.to_csv(os.path.join(output_dir, 'detection_metrics.csv'))
        
        # 메트릭스 출력
        print("\nDetection Metrics by Category:")
        print(metrics_df)
        
        # 전체 평균 출력
        if not metrics_df.empty:
            print("\nOverall Metrics:")
            print(f"Average Confidence: {metrics_df['avg_confidence'].mean():.4f}")
            print(f"Total Detections: {metrics_df['detections'].sum()}")
        
        print(f"\nResults saved in: {output_dir}")
        return results, metrics_df

    def calculate_metrics(self, major_category_detections):
        """대분류별 메트릭스 계산"""
        metrics = {}
        
        for major_class, detections in major_category_detections.items():
            if detections:
                confidences = [d['confidence'] for d in detections]
                metrics[major_class] = {
                    'detections': len(detections),
                    'avg_confidence': np.mean(confidences),
                    'max_confidence': np.max(confidences),
                    'min_confidence': np.min(confidences)
                }
        
        return metrics

def main():
    # 경로 설정
    model_path = r"C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241122_201936\waste_detection_model\weights\best.pt"
    image_path = r"C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\images\2024110181236-36-423815-127-406915-2_jpg.rf.cb26658c1c026e38efb69aad7d250449.jpg"
    save_dir = r"C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\results_20241122_201936\waste_detection_model\weights"
    
    # 객체 탐지기 초기화 및 실행
    detector = WasteDetector(model_path, conf_threshold=0.25)
    results, metrics = detector.detect_and_classify(image_path, save_dir)
    
    print(f"Detection completed. Results saved in: {save_dir}")

if __name__ == "__main__":
    main()


image 1/1 C:\Users\test\Desktop\waste detection.v4-add-pp-bag-8-1-1-.coco\train\images\2024110181236-36-423815-127-406915-2_jpg.rf.cb26658c1c026e38efb69aad7d250449.jpg: 640x640 2 Drawer units, 8 PP bags, 1 lumber, 1 waste pile, 35.7ms
Speed: 2.0ms preprocess, 35.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

Detections in image 1:

PP bag:
  - Confidence: 0.98
  - Confidence: 0.95
  - Confidence: 0.93
  - Confidence: 0.81
  - Confidence: 0.50
  - Confidence: 0.43
  - Confidence: 0.35
  - Confidence: 0.33

Large Waste Items:
  - Confidence: 0.48
  - Confidence: 0.40
  - Confidence: 0.40

General Waste:
  - Confidence: 0.39

Detection Metrics by Category:
                   detections  avg_confidence  max_confidence  min_confidence
PP bag                    8.0        0.659655        0.975923        0.332756
Large Waste Items         3.0        0.429025        0.480693        0.401839
General Waste             1.0        0.385941        0.385941        0.385941

O